In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException

from bs4 import BeautifulSoup
import time
import csv

In [2]:
#메디게이트 사이트의 뉴스 기사를 크롤링함
site = 'https://www.medigatenews.com/section/medicine/list?canada=&page_no=10&page_size=100'

#csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
f = open('medi_gate.csv', 'w',encoding='utf-8')
#파일 객체를 생성
wr = csv.writer(f)


In [3]:
#현재 PC에 설치한 chromedriver의 위치를 잡아준다.
chrome_driver = 'D:/study/Python/SW/chromedriver.exe'

#chrome으로 driver를 실행
driver = webdriver.Chrome(chrome_driver)

#조건 성립 시간을 위한 wait 객체 설정
wait = WebDriverWait(driver,20)

#driver의 브라우저 크기를 최대화
driver.maximize_window()
#최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

#웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
driver.implicitly_wait(3)

#미리 설정해둔 공공보건 포털사이트로 이동한다.
driver.get(site)

#기사의 제목, 내용, 생성날짜를 저장할 변수를 설정한다.
name=""
content=""
date=""

page_now = 10

row = 1
while True:
    news_list = driver.find_elements_by_css_selector("div.contwrap.newslst div.contlst")
    time.sleep(1)
    list_num = len(news_list)
    while True:
        try:
            news = driver.find_element_by_css_selector("div.contwrap.newslst div.contlst:nth-child(" + str(list_num) + ")") 
            date = news.find_element_by_css_selector("p.date span:first-child").text
            name = news.find_element_by_css_selector("p.titxt").text
            news.find_element_by_css_selector("a").send_keys(Keys.CONTROL + "\n")
            driver.switch_to_window(driver.window_handles[1])
            time.sleep(1)
            content = driver.find_element_by_css_selector("div.contarea").text
            wr.writerow([row,name, date, content])
            row += 1
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
            if(list_num == 1):
                break
            list_num -= 1
        except NoSuchElementException:
            if(list_num == 1):
                break
            list_num -= 1
    page_now -= 1
    if(page_now == 0):
        break
    page = 'https://www.medigatenews.com/section/medicine/list?canada=&page_no='+ str(page_now) + '&page_size=100'
    driver.get(page)


KeyboardInterrupt: 